In [1]:
import sys
sys.path.insert(0,'/home/olgan96/.env/lib/python3.11/site-packages/')
# https://github.com/UnixJunkie/mol2ecfp4/blob/master/ecfp4.py

In [2]:
import pandas as pd

In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem

In [4]:
import pandas as pd
import numpy as np

### Filter drugs

In [5]:
df_drugs = pd.read_parquet('../../NO_BACKUP/tahoe/drug_meta.parquet')

In [25]:
df_comp = pd.read_parquet('../../NO_BACKUP/pharmacodb/compounds_pharmdb.parquet')

In [26]:
df_comp

,id,name,uid,inchikey,smiles,pubchem,fda_status,chembl,reactome
0,1,Ether-20,PDBC03231,,,None,NA,None,NA
1,2,"((((2-(Hydroxy(oxido)amino)-4,5-dimethoxybenzy...",PDBC03232,JDYVLIMGUYDXMH-UHFFFAOYSA-N,COC1=C(C=C(C(=C1)COC(=O)N(CC(=O)O)C2=CC=CC=C2)...,375630,NA,CHEMBL1974689,NA
2,3,((((4-Cyanophenyl)imino)(1-naphthylamino)methy...,PDBC03233,ZIBNPJVIVGAAMD-UHFFFAOYSA-N,C1=CC=C2C(=C1)C=CC=C2NC(=NCC(=O)O)NC3=CC=C(C=C...,378568,NA,CHEMBL1997627,NA
3,4,((((4-Cyanophenyl)imino)(2-naphthylamino)methy...,PDBC03234,FRSPBOBJXMHBDQ-UHFFFAOYSA-N,C1=CC=C2C=C(C=CC2=C1)NC(=NCC(=O)O)NC3=CC=C(C=C...,378574,NA,CHEMBL1984690,NA
4,5,(((1-(Benzylthio)cyclopentyl)thio)methyl)benzene,PDBC03235,PQHMYYKJZLCSJD-UHFFFAOYSA-N,C1CCC(C1)(SCC2=CC=CC=C2)SCC3=CC=CC=C3,379229,NA,CHEMBL1966801,NA
...,...,...,...,...,...,...,...,...,...
56144,56145,{[2-(Bis-methoxycarbonylmethyl-amino)-cyclohex...,PDBC57236,FBUCXDSCOIEFJY-UHFFFAOYSA-N,COC(=O)CN(CC(=O)NCCCBr)C1CCCCC1N(CC(=O)OC)CC(=...,384578,NA,CHEMBL1975112,NA
56145,56146,"{[2-({[4-Amino-6-(5-methyl-4,5-dihydro-1H-pyra...",PDBC57237,BSQXMHXNJIAHEO-UHFFFAOYSA-N,CC1CC=NN1C2=NC(=NC(=N2)NS(=O)(=O)C3=C(C=C(C(=C...,378491,NA,CHEMBL1986163,NA
56146,56147,"{[2-({[4-Amino-6-(dimethylamino)-1,3,5-triazin...",PDBC57238,DIEODVBPTSOULE-UHFFFAOYSA-N,CC1=CC(=C(C=C1Cl)SC(C2=CC=CC=C2)C(=O)O)S(=O)(=...,378488,NA,CHEMBL1964979,NA
56147,56148,"{[2-({[4-Amino-6-(dimethylamino)-1,3,5-triazin...",PDBC57239,UYBJQMWOCWMWQV-UHFFFAOYSA-N,CC1=CC(=C(C=C1Cl)SCC(=O)O)S(=O)(=O)NC2=NC(=NC(...,378486,NA,CHEMBL2000998,NA


In [7]:
df_drugs = df_drugs[~df_drugs['pubchem_cid'].isna()].reset_index(drop=True)

In [8]:
df_drugs['pubchem_cid'] = df_drugs['pubchem_cid'].astype(int).astype(str)

In [9]:
comp_overlap = df_drugs[df_drugs['pubchem_cid'].isin(df_comp['pubchem'])].reset_index(drop=True)

In [10]:
from rdkit.Chem import rdFingerprintGenerator
fp_diameter = 4
fp_radius = fp_diameter / 2
rdkgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)
fps = []
cfps = []

for smile in comp_overlap['canonical_smiles'].values:
    if not smile is None:
        mol = Chem.MolFromSmiles(smile)
        fp = rdkgen.GetFingerprintAsNumPy(mol)
        cfp = rdkgen.GetCountFingerprintAsNumPy(mol)
        fps.append(fp)
        cfps.append(cfp)
    else:
        fps.append(None)
        cfps.append(None)

In [11]:
df_fps = pd.DataFrame(np.array(fps))

In [12]:
rename_fps_col = {}
for i in range(2048):
    rename_fps_col[i] = 'fps_' + str(i)

In [13]:
df_fps = df_fps.rename(columns=rename_fps_col)

In [14]:
df_fps['pubchem_cid'] = comp_overlap['pubchem_cid']

In [15]:
comp_overlap

,drug,targets,moa-broad,moa-fine,human-approved,clinical-trials,gpt-notes-approval,canonical_smiles,pubchem_cid
0,Bortezomib,PSMB5,inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma and mantle cell ...,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,387447
1,Ixazomib,PSMB5,inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma treatment.,B(C(CC(C)C)NC(=O)CNC(=O)C1=C(C=CC(=C1)Cl)Cl)(O)O,25183872
2,Fumaric acid,None,inhibitor/antagonist,unclear,yes,yes,Used in psoriasis treatment as part of fumaric...,C(=CC(=O)O)C(=O)O,444972
3,Hydroxyurea,"RRM2, RRM1",inhibitor/antagonist,DNA synthesis/repair inhibitor,yes,yes,Approved for sickle cell anemia and certain ca...,C(=O)(N)NO,3657
4,Darinaparsin,None,unclear,unclear,no,yes,Used in clinical trials for certain cancers only.,C[As](C)SCC(C(=O)NCC(=O)O)NC(=O)CCC(C(=O)O)N,11683005
...,...,...,...,...,...,...,...,...,...
155,Vismodegib,"SMO, ABCB1, ABCG2",inhibitor/antagonist,Sonic inhibitor,yes,yes,Approved for treating basal cell carcinoma.,CS(=O)(=O)C1=CC(=C(C=C1)C(=O)NC2=CC(=C(C=C2)Cl...,24776445
156,Busulfan,None,unclear,DNA synthesis/repair inhibitor,yes,yes,Approved for use in bone marrow transplantation.,CS(=O)(=O)OCCCCOS(=O)(=O)C,2478
157,Trametinib (DMSO_TF solvate),"MAP2K1, MAP2K2",inhibitor/antagonist,MEK inhibitor,yes,yes,"Approved for use in cancer treatment (e.g., me...",CC1=C2C(=C(N(C1=O)C)NC3=C(C=C(C=C3)I)F)C(=O)N...,11707110
158,Pitavastatin (Calcium),HMGCR,inhibitor/antagonist,unclear,yes,yes,"Approved for lowering cholesterol, part of sta...",C1C(C1)C2=NC3=CC=CC=C3C(=C2/C=C/[C@@H](O)C[C@...,5282451


In [16]:
df_fps

,fps_0,fps_1,fps_2,fps_3,fps_4,fps_5,fps_6,fps_7,fps_8,fps_9,...,fps_2039,fps_2040,fps_2041,fps_2042,fps_2043,fps_2044,fps_2045,fps_2046,fps_2047,pubchem_cid
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,387447
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,25183872
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,444972
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3657
4,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11683005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,24776445
156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2478
157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,11707110
158,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5282451


In [17]:
df_fps['drug'] = comp_overlap['drug']

In [18]:
df_fps['smiles'] = comp_overlap['canonical_smiles']

In [ ]:
df_fps.merge()

In [19]:
df_cfps = pd.DataFrame(np.array(cfps))

In [20]:
rename_cfps_col = {}
for i in range(2048):
    rename_cfps_col[i] = 'cfps_' + str(i)
df_cfps = df_cfps.rename(columns=rename_cfps_col)

In [21]:
df_fps['drug'] = comp_overlap['drug']
df_fps['smiles'] = comp_overlap['canonical_smiles']
df_cfps = pd.DataFrame(np.array(cfps))

In [22]:
rename_cfps_col = {}
for i in range(2048):
    rename_cfps_col[i] = 'cfps_' + str(i)
df_cfps = df_cfps.rename(columns=rename_cfps_col)

In [24]:
df_fps.to_parquet('../../NO_BACKUP/tahoe/comp_overlap_fps.parquet', index=False)
df_cfps.to_parquet('../../NO_BACKUP/tahoe/comp_overlap_cfps.parquet', index=False)